In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from hydra import initialize, initialize_config_dir, initialize_config_module, compose
from omegaconf import OmegaConf
from common.parser import parse_cfg
from common import MODEL_SIZE, TASK_SET
import re
from pathlib import Path
import torch

from common.buffer import ReplayBuffer

with initialize(version_base=None, config_path='.'):
    cfg = compose(config_name='config.yaml')

    # Logic
    for k in cfg.keys():
        try:
            v = cfg[k]
            if v == None:
                v = True
        except:
            pass

    # Algebraic expressions
    for k in cfg.keys():
        try:
            v = cfg[k]
            if isinstance(v, str):
                match = re.match(r"(\d+)([+\-*/])(\d+)", v)
                if match:
                    cfg[k] = eval(match.group(1) + match.group(2) + match.group(3))
                    if isinstance(cfg[k], float) and cfg[k].is_integer():
                        cfg[k] = int(cfg[k])
        except:
            pass

    # Convenience
    cfg.work_dir = '.'
    cfg.task_title = cfg.task.replace("-", " ").title()
    cfg.bin_size = (cfg.vmax - cfg.vmin) / (cfg.num_bins-1) # Bin size for discrete regression

    # Model size
    if cfg.get('model_size', None) is not None:
        assert cfg.model_size in MODEL_SIZE.keys(), \
            f'Invalid model size {cfg.model_size}. Must be one of {list(MODEL_SIZE.keys())}'
        for k, v in MODEL_SIZE[cfg.model_size].items():
            cfg[k] = v
        if cfg.task == 'mt30' and cfg.model_size == 19:
            cfg.latent_dim = 512 # This checkpoint is slightly smaller

    # Multi-task
    cfg.multitask = cfg.task in TASK_SET.keys()
    if cfg.multitask:
        cfg.task_title = cfg.task.upper()
        # Account for slight inconsistency in task_dim for the mt30 experiments
        cfg.task_dim = 96 if cfg.task == 'mt80' or cfg.model_size in {1, 317} else 64
    else:
        cfg.task_dim = 0
    cfg.tasks = TASK_SET.get(cfg.task, [cfg.task])



/home/j/workspace/tdmpc2/venv/lib/python3.9/site-packages/torchrl/data/replay_buffers/samplers.py:37: UserWarning: Failed to import torchrl C++ binaries. Some modules (eg, prioritized replay buffers) may not work with your installation. If you installed TorchRL from PyPI, please report the bug on TorchRL github. If you installed TorchRL locally and/or in development mode, check that you have all the required compiling packages.
  warnings.warn(EXTENSION_WARNING)


In [3]:
from envs import make_env
env = make_env(cfg)

Pusht force sparse reward:  False


In [19]:
import cv2
import numpy as np
from pynput import mouse
import time

from tensordict.tensordict import TensorDict
def to_td(obs, env, action=None, reward=None):
    """Creates a TensorDict for a new episode."""
    if isinstance(obs, dict):
        obs = TensorDict(obs, batch_size=(), device='cpu')
    else:
        obs = obs.unsqueeze(0).cpu()
    if action is None:
        action = torch.full_like(env.rand_act(), float('nan'))
    if reward is None:
        reward = torch.tensor(float('nan'))
    td = TensorDict(dict(
        obs=obs,
        action=action.unsqueeze(0),
        reward=reward.unsqueeze(0),
    ), batch_size=(1,))
    return td

from common.buffer import Buffer
buffer = Buffer(cfg)

Pointer moved to (549, 461) -> (-0.902, -0.19499999999999995)
Pointer moved to (548, 463) -> (-0.904, -0.18500000000000005)
Pointer moved to (548, 470) -> (-0.904, -0.15000000000000002)
Pointer moved to (546, 482) -> (-0.908, -0.08999999999999997)
Pointer moved to (545, 493) -> (-0.91, -0.03500000000000003)
Pointer moved to (541, 504) -> (-0.918, 0.020000000000000018)
Pointer moved to (540, 511) -> (-0.92, 0.05499999999999994)
Pointer moved to (536, 515) -> (-0.928, 0.07499999999999996)
Pointer moved to (534, 517) -> (-0.9319999999999999, 0.08499999999999996)
Pointer moved to (533, 519) -> (-0.9339999999999999, 0.09499999999999997)
Pointer moved to (532, 520) -> (-0.9359999999999999, 0.10000000000000009)
Pointer moved to (532, 522) -> (-0.9359999999999999, 0.1100000000000001)
Pointer moved to (531, 523) -> (-0.938, 0.11499999999999999)
Pointer moved to (530, 525) -> (-0.94, 0.125)
Pointer moved to (529, 526) -> (-0.942, 0.1299999999999999)
Pointer moved to (529, 528) -> (-0.942, 0.1399

In [20]:
global next_action
next_action = np.array([0,0])

minx, maxx = 500, 1500
miny, maxy = 300, 700
def on_move(x, y):
    if x < minx or x > maxx or y < miny or y > maxy: return np.array([0, 0])
    
    xnorm = (x - minx) / (maxx - minx)
    ynorm = (y - miny) / (maxy - miny)

    xnorm = max(0, min(1, xnorm))
    ynorm = max(0, min(1, ynorm))

    xnorm = 2 * xnorm - 1
    ynorm = 2 * ynorm - 1

    print(f'Pointer moved to {(x, y)} -> {xnorm, ynorm}')
    global next_action
    next_action = np.array([xnorm, ynorm])

# Create a listener
listener = mouse.Listener(on_move=on_move)

# Start the listener
listener.start()
cv2.destroyAllWindows()
obs, done, ep_reward, t = env.reset(), False, 0, 0
tds = [to_td(obs, env)]
eps = 0; ts = 0
while eps < 4:
    # action = torch.Tensor(env.action_space.sample())
    action = torch.Tensor(next_action)
    obs, reward, done, info = env.step(action)
    tds.append(to_td(obs, env, action, reward))

    if done or ts >= 300:
        eps += 1
        buffer.add(torch.cat(tds))
        tds = [to_td(obs, env)]


    img = obs.detach().cpu().numpy()
    # Step 1: Reshape the stack into separate images
    img = img.transpose(1, 2, 0)
    # reshaped = np.hstack([img[:,:,i*3:(i*3)+3] for i in range(3)])
    reshaped = img[:, :, -3:]
    reshaped = cv2.resize(reshaped, (reshaped.shape[1] * 3, reshaped.shape[0] * 3), interpolation=cv2.INTER_NEAREST)

    # Step 3: Display using OpenCV
    cv2.imshow('row', reshaped)
    cv2.waitKey(100)
    ts += 1

cv2.destroyAllWindows()
listener.stop()

Pointer moved to (755, 413) -> (-0.49, -0.43500000000000005)
Pointer moved to (755, 413) -> (-0.49, -0.43500000000000005)
Pointer moved to (743, 390) -> (-0.514, -0.55)
Pointer moved to (743, 390) -> (-0.514, -0.55)
Pointer moved to (721, 340) -> (-0.558, -0.8)
Pointer moved to (721, 340) -> (-0.558, -0.8)
Pointer moved to (550, 310) -> (-0.9, -0.95)Pointer moved to (550, 310) -> (-0.9, -0.95)

Pointer moved to (528, 324) -> (-0.944, -0.88)
Pointer moved to (528, 324) -> (-0.944, -0.88)
Pointer moved to (510, 332) -> (-0.98, -0.84)
Pointer moved to (510, 332) -> (-0.98, -0.84)
Pointer moved to (505, 475) -> (-0.99, -0.125)
Pointer moved to (505, 475) -> (-0.99, -0.125)
Pointer moved to (509, 481) -> (-0.982, -0.09499999999999997)
Pointer moved to (509, 481) -> (-0.982, -0.09499999999999997)
Pointer moved to (510, 484) -> (-0.98, -0.07999999999999996)
Pointer moved to (510, 484) -> (-0.98, -0.07999999999999996)
Pointer moved to (511, 486) -> (-0.978, -0.06999999999999995)
Pointer moved 

Pointer moved to (1029, 492) -> (0.05800000000000005, -0.040000000000000036)
Pointer moved to (1029, 492) -> (0.05800000000000005, -0.040000000000000036)
Pointer moved to (1028, 491) -> (0.05600000000000005, -0.04500000000000004)
Pointer moved to (1029, 491) -> (0.05800000000000005, -0.04500000000000004)
Pointer moved to (1031, 491) -> (0.062000000000000055, -0.04500000000000004)
Pointer moved to (1032, 490) -> (0.06400000000000006, -0.050000000000000044)
Pointer moved to (1033, 489) -> (0.06600000000000006, -0.05500000000000005)
Pointer moved to (1035, 489) -> (0.07000000000000006, -0.05500000000000005)
Pointer moved to (1036, 488) -> (0.07200000000000006, -0.06000000000000005)
Pointer moved to (1037, 487) -> (0.07400000000000007, -0.06499999999999995)
Pointer moved to (1038, 486) -> (0.07600000000000007, -0.06999999999999995)
Pointer moved to (1040, 485) -> (0.08000000000000007, -0.07499999999999996)
Pointer moved to (1041, 484) -> (0.08200000000000007, -0.07999999999999996)
Pointer 

In [24]:
import os
path = os.path.expanduser("~/workspace/tdmpc2/buf.pt")

buffer.save(path)

TypeError: 'module' object is not callable

In [ ]:
# a = img.reshape(64, 64, 3, 3)
a = img
cv2.imshow('a', a)
cv2.waitKey(0)
cv2.destroyAllWindows()